Radix-2 DIT FFT was invented by Cooley and Tukey in 1965. It's widely used in digital signal processing.

It is a divide-and-conquer algorithm that recursively splits the problem into smaller subproblems. The complexity is O(N log N).

Radix-2 means that the number of points is a power of 2. DIT is Decimation In Time, means that the input is decimated in time domain. we will show why we need FFT and how can we do Radix-2 DIT FFT.

### 1. Background of the problem
#### 1.1 Example

One representation of the polynomial is using coefficient form. Such as:

$$f(x) = a_0 + a_1 x + a_2 x^2 + a_3 x^3$$

$$g(x) = b_0 + b_1 x + b_2 x^2 + b_3 x^3$$

Normally, if you want to compute the convolution of $f(x)$ and $g(x)$, you would need to compute the following:

$$h(x) = f(x) * g(x) = (a_0 + a_1 x + a_2 x^2 + a_3 x^3) * (b_0 + b_1 x + b_2 x^2 + b_3 x^3) = c_0 + c_1 x + c_2 x^2 + c_3 x^3 + c_4 x^4 + c_5 x^5 + c_6 x^6$$

This is a polynomial multiplication, and the complexity is O(N^2).Can we lower the complexity?

Before we answer this question, let's take a look at another representation of the polynomial.
#### 1.2 Coefficient form and evaluation form
A polynomial in the variable x over an algebraic field F represents a function f(x) as a formal sum:
$$f(x)=\sum_{i=1}^n a_i*x^i$$
such as the formula above:$f(x) = a_0 + a_1 x + a_2 x^2 + a_3 x^3$, We call the values $a_0,a_1,a_2,a_3$ the coefficients of the polynomial.

Another form of a polynomial is called evaluation form, that means a point-value representation of a polynomial f(x) of degree-bound n is a set of n point-value pairs:
$${(x_0,y_0),(x_1,y_1)...(x_{n-1},y_{n-1})}$$

such as the polynomial $f(x)$ and $g(x)$ above:

$$f(x) = (x_0, f(x_0)), (x_1, f(x_1)), (x_2, f(x_2)), (x_3, f(x_3))$$

$$g(x) = (x_0, g(x_0)), (x_1, g(x_1)), (x_2, g(x_2)), (x_3, g(x_3))$$
Then, we want the value of points on $h(x)$: $h(x_0)=f(x_0)*g(x_0)$

### 1.3 Do multiplication by evaluation

So we can see, if we want compute multiplication of two polynomial,the complexity of this is only O(N) by evaluation form. But we need expand the points first. 4 points are enough to represent a polynomial of degree 3. From previous section, we know that $h(x)$ is a polynomial. of degree 6. So we need at least 7 points to represent $h(x)$.

So the steps of polynomial multiplication in point-value form is as follows:

1. Extend the fields of $f(x)$ and $g(x)$ to at least 7 points.
2. Compute the point-value form of $f(x)$ and $g(x)$.
3. Multiply the point-value form of $f(x)$ and $g(x)$.

Let's take a look at the steps in detail.
1. Extend the fields of $f(x)$ and $g(x)$ to at least 7 points. We use 8 points for convenience. Later we will explain why we need to extend the fields and why we use 8 points.
So new fields are $x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7$.

2. Compute the point-value form of $f(x)$ and $g(x)$.
$f(x) = (x_0, f(x_0)), (x_1, f(x_1)), (x_2, f(x_2)), (x_3, f(x_3)), (x_4, f(x_4)), (x_5, f(x_5)), (x_6, f(x_6)), (x_7, f(x_7))$
$g(x) = (x_0, g(x_0)), (x_1, g(x_1)), (x_2, g(x_2)), (x_3, g(x_3)), (x_4, g(x_4)), (x_5, g(x_5)), (x_6, g(x_6)), (x_7, g(x_7))$

3. Multiply the point-value form of $f(x)$ and $g(x)$.
$h(x) = (x_0, f(x_0) * g(x_0)), (x_1, f(x_1) * g(x_1)), (x_2, f(x_2) * g(x_2)), (x_3, f(x_3) * g(x_3)), (x_4, f(x_4) * g(x_4)), (x_5, f(x_5) * g(x_5)), (x_6, f(x_6) * g(x_6)), (x_7, f(x_7) * g(x_7))$

If we can always compute polynomial multiplication in point-value form in O(N) time, we can use this to speed up the polynomial multiplication.

However, there are two problems:
1. How to convert a polynomial from coefficient form to point-value form?
2. How to convert a polynomial from point-value form to coefficient form?
The first problem is called polynomial interpolation. The second problem is called polynomial evaluation.

### 2. DFT 
#### 2.1 Compute directly
We can certainly do the polynomial form transform computation directly, actually it's like a matrix operation.
Let's use vectors to express this: 

$\vec{f}$=$
\begin{pmatrix}
f(x_0)\\
f(x_1)\\
\cdots\\
f(x_{n-1}) 
\end{pmatrix}
$=
$
\left(\begin {array}{c}
1 &x_0 &{x_0}^2 &{\cdots} &{x_0^{n-1}}\\
1 &x_1 &{x_1}^2 &{\cdots} &{x_1^{n-1}} \\
&{\cdots} &{\cdots} &{\cdots} &{\cdots} \\
1 &x_{n-1} &x_{n-1}^2 &{\cdots} &{x_{n-1}^{n-1}} \\
\end{array}\right)
$*$
\begin{pmatrix}
a_1\\
a_2\\
\cdots\\
a_{n-1} 
\end{pmatrix}
$

We donate the Vandermonde matrix of x as M, that is: $$\vec{f}=M*\vec{a}$$

then inversely, we compute the coffecients $\vec{a}$ like this: $$\vec{a}=M^{-1}*\vec{f}$$
#### 2.2 DFT
DFT is a mathematical tool for converting discrete signals from the time domain to the frequency domain. It is a discrete version of the Fourier transform and is particularly suitable for processing discrete and finite length signals.
Recall the problem above,this time we evaluate f(x) at complex nth roots of unity, instead of random points $x_0,x_1...x_{n-1}$.

A complex nth root of unity is a complex number $\omega$ such that $\omega^n=1$.We use n complex nth roots of unity,then $\omega_{n}=e^{2\pi ik/n}$. <br>
The n complex nth roots of unity is $\omega_{n}^0,\omega_{n}^1,\omega_{n}^2...\omega_{n}^{n-1}$, we can see the n complex roots of unity are equally spaced around the circle of unit radius centered at the origin of the complex plane.<br>

<div style="text-align: center;">
    <img src="https://i.imgur.com/vus7Ml6.png" alt="nth roots of unity" style="width: 30%;"/>
</div>

Further,if n is power of 2,we can get $\omega_{n}^{i}=-\omega_{n}^{n/2+i}$, the points on the circle are paired,such as: <br>
$\omega_{n}^{1}=-\omega_{n}^{5},\omega_{n}^{2}=-\omega_{n}^{6},\omega_{n}^{3}=-\omega_{n}^{5},\omega_{n}^{7}=-\omega_{n}^{5},\omega_{n}^{4}=-\omega_{n}^{0}=-1$

With this condition attached, on this basis, we can start to do FFT!

One more thing, due to the imprecision issues associated with using complex numbers, we utilize finite fields as a replacement for expressing values.

### 3. FFT 
#### 3.1 Overview of the Framework
By using a method known as the fast Fourier transform (FFT), which takes advantage of the special properties of the complex roots of unity, we can compute DFT in time $O(nlogn)$, as opposed to the $O(n^2)$ time of the straightforward method.
The FFT method employs a divide-and-conquer strategy, using the even-indexed and odd-indexed coefficients of $f(x)$ separately to define the two new polynomials $f_{even}(x)$ and $f_{odd}(x)$ of degree-bound n/2. Note that here n is power of 2:

$$f(x)=f_{even}(x^2)+x*f_{odd}(x^2),\quad f(-x)=f_{even}(x^2)-x*f_{odd}(x^2)\quad \forall x \in \{1,w,w^2...w^n\}$$ 

Here we can see, the split function $f_{even}(x^2)$ and $f_{odd}(x^2)$ are functions of $x^2,\quad \forall x^2 \in \{1,w^2,w^4...w^n\}$,
if we can get the evaluation of $f_{even}(x^2)$ and $f_{odd}(x^2)$, then we can get the evaluation of $f(x)$. so we can do this recursively to get next layer split fuction.

Next, we will show the FFT algorithm step by step through an example.

Here is the simple case:given the coeffecients of $f(x)$ and $g(x)$，compute $h(x)=f(x)*g(x)$
$$f(x)=1+2x+3x^2+4x^3$$
$$g(x)=4+5x+6x^2+7x^3$$

#### 3.2 FinteField & Domain
##### FinteField
Finite fields are algebraic structures that consist of a finite number of elements, and they play a crucial role in various areas of mathematics, computer science, and cryptography.
Here is the python implementation of finte field that provides functions for basic arithmetic operations (addition, subtraction, multiplication, division, exponentiation).

In [3]:
class Field:
    modulus = None  # Class-level variable to store the modulus of the field

    @classmethod
    def set_modulus(cls, p):
        if p <= 0:
            raise ValueError("Modulus must be a positive integer.")
        cls.modulus = p #Set the modulus for the finite field

    def __init__(self, value):
        if Field.modulus is None:
            raise ValueError("Modulus is not set. Call Field.set_modulus(p) first.")
        self.value = value % Field.modulus  # Ensure the value is within the field

    def __repr__(self):
        return str(self.value)

    def __eq__(self, other):
        if isinstance(other, Field):
            return self.value == other.value
        return False

    def __add__(self, other):
        if isinstance(other, Field):
            return Field(self.value + other.value)
        raise ValueError("Cannot add elements from different fields.")

    def __sub__(self, other):
        if isinstance(other, Field):
            return Field(self.value - other.value)
        raise ValueError("Cannot subtract elements from different fields.")

    def __mul__(self, other):
        if isinstance(other, Field):
            return Field(self.value * other.value)
        elif isinstance(other, int):
            return Field(self.value * other)
        raise ValueError("Cannot multiply Field with non-integer or non-Field.")
    
    def __rmul__(self, other):
        return self.__mul__(other)

    def __truediv__(self, other):
        if isinstance(other, Field):
            return self * other.inverse()
        elif isinstance(other, int):
            return self / Field(other)
        raise ValueError("Cannot divide Field by non-integer or non-Field.")

    def __pow__(self, exponent):
        if exponent < 0:
            # Negative exponent: compute the inverse first
            return self.inverse() ** -exponent
        result = pow(self.value, exponent, Field.modulus)
        return Field(result)

    def inverse(self):
        t, new_t = 0, 1
        r, new_r = Field.modulus, self.value
        while new_r != 0:
            quotient = r // new_r
            t, new_t = new_t, t - quotient * new_t
            r, new_r = new_r, r - quotient * new_r
        if r > 1:
            raise ValueError(f"{self.value} has no multiplicative inverse in F_{Field.modulus}.")
        return Field(t % Field.modulus)

    def __neg__(self):
        return Field(-self.value)

    # Python's right-hand operations
    __radd__ = __add__
    __rsub__ = lambda self, other: Field(other) - self
    __rmul__ = __mul__


def test_field():
    Field.set_modulus(17) # Create elements in the finite field F_17
    a = Field(5)
    b = Field(10)
    # Perform operations
    print(a + b)  # Addition: 15
    print(a - b)  # Subtraction: 12
    print(a * b)  # Multiplication: 16
    print(a / b)  # Division: 9
    print(a ** 3)  # Exponentiation: 15
    print(b.inverse())  # Multiplicative inverse: 12

test_field()

15
12
16
9
6
12


##### Domain
First we need choose a domain for the evaluate points, here we will see how the Domain get halved by each layer of FFT.
Since the following deals with finite field operations, we need to install the galois library first.
we choose p=17, and choose $\omega=2$ as the 8th root of  unity. <br>
First Layer, Domain ${D_1}=<\omega>=\{\omega_{0},\omega_{1}...\omega_{7}\}={1,2,4,8,16,15,13,9}，|D_1|=8$ <br>
Second Layer, Domain ${D_2}=<\omega^2>=\{\omega_{0},\omega_{2},\omega_{4},\omega_{6}\}={1,4,16,13}，|D_2|=4$ <br>
Third Layer, Domain ${D_3}=<\omega^4>=\{\omega_{0},\omega_{4}\}={1,16}，|D_3|=2$ <br>
Last Layer, Domain ${D_4}=<\omega^8>=\{1\}，|D_3|=1$ <br>

In [4]:
Field.set_modulus(17)
w = Field(2)
order=8

while order!=0:
    Domain=[w**i for i in range(order)]
    print(Domain)
    w=w**2
    order=order//2

[1, 2, 4, 8, 16, 15, 13, 9]
[1, 4, 16, 13]
[1, 16]
[1]


#### 3.3 Split and Recursion
We take the function f(x) above, split it into two parts:
$$f(x)=1+2x+3x^2+4x^3=f_{even}(x^2)+x*f_{odd}(x^2)=(1+3x^2)+x(2+4x^2)\quad \forall x \in \{1,2,4,8,16,15,13,9\}$$
We should first compute the split function:
$$f_{even}(x^2)=1+3x^2,f_{odd}(x^2)=2+4x^2 \quad \forall x^2 \in \{1,4,16,13\}$$
For this two functions are simple,so here we can directly get the result,otherwise shold do split recursively until the length of Domain is one, then recursively cobmine them then get evaluations of $f(x)$.<br>

Here is the steps:<br>
1.we get evaluation of $f_{even}(x^2)$ and $f_{odd}(x^2)$ on ${1,2,4,8}$<br>
2.then we can get evaluation of $f(x)$ on ${1,2,4,8}$ by combination: $f_{even}(x^2)+x*f_{odd}(x^2)$<br>
3.then we can get evaluation of $f(x)$ on ${-1,-2,-4,-8}$ (actually,that is ${16,15,13,9}$) by combination: $f_{even}(x^2)-x*f_{odd}(x^2)$<br>

We can see the code of fft and ifft function.
we use fft function to transform coefficients form to evaluation form, and use ifft reverse back.

In [ ]:
def get_domain(p,generator):
        Field.set_modulus(p)
        w = Field(generator)
        domain=[]
        for i in range(p):
            k=w**i
            if i!=0 and k==Field(1):
                return domain
            else:
                domain.append(k)
        return domain

def halve_domain(domain, preserve_length=False):
        new_length = len(domain) if preserve_length else len(domain)//2
        return [x**2 for x in domain[:new_length]]

# fft: from coefficients to evaluations
def fft(vals, domain):
        if (len(domain) & (len(domain) - 1)) != 0:
                raise ValueError("Domain length must be a power of 2.")  

        if len(vals) < len(domain):
                if len(vals) == 0:  
                        zero = Field(0)
                else:  
                        zero = vals[0] - vals[0]
                vals = vals + [zero] * (len(domain) - len(vals))

        if len(vals) == 1:
            return vals
        half_domain = halve_domain(domain)
        f0 = fft(vals[::2], half_domain)
        f1 = fft(vals[1::2], half_domain)
        left = [L+x*R for L,R,x in zip(f0, f1, domain)]
        right = [L-x*R for L,R,x in zip(f0, f1, domain)]
        return left+right

# ifft: from evaluations to coefficients
def inv_fft(vals, domain):
        if (len(domain) & (len(domain) - 1)) != 0:
                raise ValueError("Domain length must be a power of 2.")
        
        if len(vals) < len(domain):
                if len(vals) == 0:  
                        zero = Field(0)
                else:  
                        zero = vals[0] - vals[0]
                vals = vals + [zero] * (len(domain) - len(vals))

        if len(vals) == 1:
            return vals
        half_domain = halve_domain(domain)
        left = vals[:len(domain)//2]
        right = vals[len(domain)//2:]
        f0 = [(L+R)/2 for L,R in zip(left, right)]
        f1 = [(L-R)/(2*x) for L,R,x in zip(left, right, domain)]
        o = [0] * len(domain)
        o[::2] = inv_fft(f0, half_domain)
        o[1::2] = inv_fft(f1, half_domain)
        return o

[1, 2, 4, 8, 16, 15, 13, 9]
[1, 2, 3, 4]
[10, 15, 7, 13, 15, 11, 6, 16]
[1, 2, 3, 4, 0, 0, 0, 0]


now we can use the function fft and inv-fft to compute evaluations and coefficients of polynomial.<br>
For $f(x)=1+2x+3x^2+4x^3$,$g(x)=5+6x+7x^2+8x^3$, we can get $h(x)=f(x)*g(x)=5+16x+9x^3+10x^4+x^5+15x^6$<br>
If you compute directly, you can get the same result.

In [33]:
# set the FieldElement and generator. we choose 17 as modulus, choose 2 as the generator.
p=17
generator=2

# a small exmaple: coefficients is [1,2,3,4]
domain=get_domain(p,generator)
original_coef_fx=[1,2,3,4]
coef_fx=[Field(x) for x in original_coef_fx]
print("domian is:",domain)
print("coefficients of f(x) is:",coef_fx)
# do fft to get the evaluations,and do ifft to get coefficients inversely
eval_fx=fft(coef_fx,domain)
print("evaluations of f(x) is:",eval_fx)
check_coef_fx=inv_fft(eval_fx,domain)
print("check coefficients of f(x):",check_coef_fx) # check for the coefficients backward calculated by inv-fft

# compute evaluation g(x)
original_coef_gx=[5,6,7,8]
coef_gx=[Field(x) for x in original_coef_gx]
eval_gx=fft(coef_gx,domain)
print("evaluations of h(x) is:",eval_gx)
# compute h(x)=f(x)*g(x)
eval_hx = [f * g for f, g in zip(eval_fx, eval_gx)]
print("evaluations of h(x) is:",eval_hx)
# get coefficients of h(x) by inv-fft
coef_hx=inv_fft(eval_hx,domain)
print("coefficients of f(x) is:",coef_hx)

domian is: [1, 2, 4, 8, 16, 15, 13, 9]
coefficients of f(x) is: [1, 2, 3, 4]
evaluations of f(x) is: [10, 15, 7, 13, 15, 11, 6, 16]
check coefficients of f(x): [1, 2, 3, 4, 0, 0, 0, 0]
evaluations of h(x) is: [9, 7, 7, 7, 15, 8, 6, 15]
evaluations of h(x) is: [5, 3, 15, 6, 4, 3, 2, 2]
coefficients of f(x) is: [5, 16, 0, 9, 10, 1, 15, 0]


The following diagram shows the process of fft and inverse fft algorithm. It looks like a butterfly,and we call the factors $w^k$ twiddle factors.<br>
fft
<div style="text-align: center;">
    <img src="https://hackmd.io/_uploads/HkfWuHLIye.png" alt="nth roots of unity" style="width: 85%;"/>
</div>
Inv-fft
<div style="text-align: center;">
    <img src="https://hackmd.io/_uploads/H1UWOSIU1x.png" alt="nth roots of unity" style="width: 85%;"/>
</div>